# Prepare Riksdagens Protokoll for Text Analysis

## Download Text from KB-LAB API (JSON)

Use Python script `extract_json_text.py` to download `content.json` and `meta.json`. The script downloads queries packagers having "protocoll" tag (query `{ "tags": "protokoll" }`).

```bash
cd source/welfare_state_analytics
pipenv shell
cd westac/kblab
nohup python download_protocol_content_json.py >& run.log &
```

The result is stored as a Zip archive.

### Extract text from JSON and store as a corpus of individual text files 

Use the script `extract_json_text.py` to extract the text from the JSON files.

```bash
python extract_json_text.py --source-filename ~/tmp/riksdagens_protokoll_content.zip --target-filename ~/tmp/riksdagens_protokoll_content_corpus.zip
```

The resulting Zip file contains the text files named as `prot_yyyyyy__NN.txt`. One file per protocoll.

### Vectorize the corpus to a BoW corpus (westac.VectorizedCorpus)

Use the script `vectorize_protocols.py` to create a BoW corpus.

```bash
python vectorize_protocols.py --source-filename ~/tmp/riksdagens_protokoll_content.zip --target-filename ~/tmp/riksdagens_protokoll_content_corpus.zip
```

The script calls `generate_corpus` in `westac.corpus.corpus_vectorizer`:

```python
import westac.corpus.corpus_vectorizer as corpus_vectorizer

kwargs = dict( ...vectorize arguments...)
corpus_filename = ...
output_folder = ...

corpus_vectorizer.generate_corpus(corpus_filename, output_folder=output_folder, **kwargs)

```

The resulting corpus are stored in the specified output folder in two files; a numpy file containing the DTM and a Pythin pickled file with the dictionary and a document index.

### Prepare text files for Sparv

The Sparv pipeline requires that the individual document are stored as (individual) XML files. The shell script `sparvit-to-xml` can be used to add a root tag to all text files in a Zip archive. The resulting XML files iare stored as a new Zip archive.

```bash
 sparvit-to-xml --input riksdagens_protokoll_content_corpus.zip --output riksdagens_protokoll_content_corpus_xml.zip
 ```

In [3]:
%load_ext autoreload
%autoreload 2

import os
import sys
import bokeh.plotting
import types

# from beakerx import *
# from beakerx.object import beakerx

root_folder = os.path.join(os.getcwd().split('welfare_state_analytics')[0], 'welfare_state_analytics')
corpus_folder = os.path.join(root_folder, 'data/riksdagens_protokoll')

sys.path = sys.path + [ root_folder, globals()['_dh'][-1] ]

bokeh.plotting.output_notebook(hide_banner=True)

container = types.SimpleNamespace(corpus=None, handle=None, data_source=None, data=None, figure=None)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load previously vectorized corpus

The corpus was created with the following settings:
 - Tokens were converted to lower case.
 - Only tokens that contains at least one alphanumeric character (isalnum).
 - Accents are ot removed (deacc)
 - Min token length 2 (min_len)
 - Max length not set (max_len)
 - Numerals are removed (numerals, -N)
 - Symbols are removed (symbols, -S)

Use the `vectorize_protocol` script to create a new corpus with different settings.

The corpus is processed in the following ways when loaded:

 - Exclude tokens having a total word count less than `Min count`
 - Include at most `Top count` most frequent words.
 - Group and sum up documents by year.
 - Normalize token distribution over years to 1.0
 

In [4]:
import word_trends_corpus_gui as corpus_gui

ui = corpus_gui.display_gui(corpus_folder, container=container)

In [7]:
import word_trends_gui
word_trends_gui.display_gui(container)

In [8]:
import ipyaggrid_plot
import pandas as pd
ipyaggrid_plot.display_grid(data=container.data)
#df = pd.DataFrame(data=data)
#df

,och,eller,hur
year,,,
1945,0.006424,0.008405,0.005276
1946,0.000705,0.001103,0.000380
1947,0.000000,0.000000,0.000000
1948,0.019782,0.026279,0.018525
1949,0.020122,0.025560,0.019415
1950,0.005426,0.006363,0.004798
1951,0.000000,0.000000,0.000000
1952,0.000255,0.000227,0.000118
1953,0.000000,0.000000,0.000000


Grid(columns_fit='auto', compress_data=True, export_mode='buttons', height='350px', menu={'buttons': [{'name':…

In [ ]:
dir(Grid.__init__)


def __init__(self,
              width='100%',
              height=0,
              center=False,
              theme='ag-theme-fresh',

              grid_data=[],
              grid_options={},
              grid_options_multi=[],
              columns_fit='size_to_fit',
              index=False,
              keep_multiindex=False,
              compress_data=True,

              quick_filter=False,
              export_csv=False,
              export_excel=False,
              show_toggle_delete=False,
              show_toggle_edit=False,
              paste_from_excel=False,
              export_mode='disabled',
              export_to_df=True,
              hide_grid=False,

              js_helpers_custom='',
              js_pre_helpers=[],
              js_pre_grid=[],
              js_post_grid=[],
              css_rules='',
              menu=None,
              user_params={},

              license=''):